<H4>Import libraries</H4>

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os
import sys
import random
from scipy import ndimage
import matplotlib.pyplot as plt
from sklearn.cross_validation import train_test_split

from keras.datasets import cifar10
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator

Using Theano backend.


<H4>Load CIFAR 10 dataset. Keep 10% of training data for validation</H4>

In [4]:
nb_classes = 10

def load_dataset(validation_size):    
    (X_train, y_train), (X_test, y_test) = cifar10.load_data()    
    
    y_train = np_utils.to_categorical(y_train, nb_classes)
    y_test = np_utils.to_categorical(y_test, nb_classes)
    
    X_train = X_train.astype('float32')
    X_test = X_test.astype('float32')
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=validation_size, random_state=100)
    
    print 'X_train shape: ', X_train.shape
    print X_train.shape[0], 'train samples'
    print X_val.shape[0], 'validation samples'
    print X_test.shape[0], 'test samples'
 
    return X_train, y_train, X_val, y_val, X_test, y_test

X_train, y_train, X_val, y_val, X_test, y_test = load_dataset(validation_size = 0.1)

X_train shape:  (45000L, 3L, 32L, 32L)
45000 train samples
5000 validation samples
10000 test samples


<H4>Image Data Generator for feature normalization and zca whitening</H4>

In [5]:
datagen = ImageDataGenerator(featurewise_center=True, featurewise_std_normalization=True, zca_whitening=True)
datagen.fit(X_train)

<H4>Create a model with 4 convolutional layers and 2 fully-connected layers</H4>

In [6]:
def create_network(channels, image_rows, image_cols, lr, decay, momentum):
    model = Sequential()
 
    model.add(Convolution2D(32, 3, 3, border_mode='valid', input_shape = (channels, image_rows, image_cols)))
    model.add(Activation('relu'))
    model.add(Convolution2D(32, 3, 3))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))
 
    model.add(Convolution2D(64, 3, 3, border_mode='valid'))
    model.add(Activation('relu'))
    model.add(Convolution2D(64, 3, 3))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))
 
    model.add(Flatten())
    model.add(Dense(512))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(nb_classes))
    model.add(Activation('softmax'))
    
    sgd = SGD(lr=lr, decay=decay, momentum=momentum, nesterov=True)
    
    model.compile(loss='categorical_crossentropy', optimizer=sgd)
    
    return model

model = create_network(3, 32, 32, 0.01, 1e-6, 0.9)

<H4>Fit the model and validate using the validation set</H4>

In [9]:
model.fit_generator(datagen.flow(X_train, y_train, batch_size=32), samples_per_epoch=len(X_train), nb_epoch=10,
                    validation_data = datagen.flow(X_val, y_val, batch_size = 32), nb_val_samples = 5000, verbose=1)

Epoch 1/10
45000/45000 [==============================] - 2586s - loss: 1.8190 - val_loss: 1.7049
Epoch 2/10
45000/45000 [==============================] - 2595s - loss: 1.2754 - val_loss: 1.0770
Epoch 3/10
45000/45000 [==============================] - 2561s - loss: 1.0615 - val_loss: 1.0364
Epoch 4/10
45000/45000 [==============================] - 2623s - loss: 0.9529 - val_loss: 0.8839
Epoch 5/10
45000/45000 [==============================] - 2587s - loss: 0.8849 - val_loss: 0.8852
Epoch 6/10
45000/45000 [==============================] - 2631s - loss: 0.8277 - val_loss: 0.8160
Epoch 7/10
45000/45000 [==============================] - 2677s - loss: 0.7904 - val_loss: 0.7512
Epoch 8/10
45000/45000 [==============================] - 2636s - loss: 0.7490 - val_loss: 0.7334
Epoch 9/10
45000/45000 [==============================] - 3753s - loss: 0.7203 - val_loss: 0.7116
Epoch 10/10
45000/45000 [==============================] - 2590s - loss: 0.6978 - val_loss: 0.7017


<H4>Verify the accuracy of the model on test set</H4>

In [11]:
result = model.evaluate_generator(datagen.flow(X_test, y_test, batch_size = 32), val_samples=10000)
print 'Test Loss: ', result

Test Loss:  0.708616079187
